# Create Training Sets for Chemical Interpolation Test
The goal of this test is to determine whether machine learning models are able to infer the interactions between elements that are not included in the training set. We will set up two different tests, 
1. Exclude a single quaternary. Withhold all entries that contain exclusively elements from the target 
2. Exclude a certain pair interaction. Withhold all entries that contain both elements in a certain pair

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from pymatgen import Composition
from itertools import product
import numpy as np
import pandas as pd
import os
import shutil

## Read in the OQMD dataset
We want only the lowest-energy entry at each composition

In [2]:
oqmd_data = pd.read_csv(os.path.join('..', 'data/oqmd_all.data'), delim_whitespace=True)
print('Read %d entries'%len(oqmd_data))
oqmd_data.head()
oqmd_data.shape

Read 506114 entries


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(506114, 7)

Make all of the energies numeric

In [3]:
for col in oqmd_data.columns:
    if col == 'comp': continue
    oqmd_data[col] = pd.to_numeric(oqmd_data[col], errors='coerce')

Eliminate entries with weird formation enthalpies

In [4]:
oqmd_data.query('delta_e > -20 and delta_e < 5', inplace=True)

Generate the composition object of each entry

In [5]:
oqmd_data['comp_obj'] = oqmd_data['comp'].apply(lambda x: Composition(x))

In [6]:
oqmd_data['pretty_comp'] = oqmd_data['comp_obj'].apply(lambda x: x.reduced_formula)

/usr/local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:429: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
/usr/local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:429: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
/usr/local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:429: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)


Get only the lowest-energy entry at each composition

In [7]:
oqmd_data.sort_values('delta_e', ascending=True, inplace=True)
oqmd_data.drop_duplicates('pretty_comp', keep='first', inplace=True)
print('Reduced dataset to %d entries'%len(oqmd_data))

Reduced dataset to 275701 entries


## Identify the systems with large numbers of entries
We want to find a system with a large amount of testing data

In [8]:
oqmd_data['nelems'] = oqmd_data['comp_obj'].apply(lambda x: len(x))

In [9]:
oqmd_data['system'] = oqmd_data['comp_obj'].apply(lambda x: "-".join([y.symbol for y in x]))

Get the top-10 most frequent systems

In [10]:
oqmd_data['system'].value_counts()[:10]

Mn-Na-O    20
O-V        18
O-Ti       18
H-O-V      17
Li-O-V     17
Fe-Na-O    17
C-H-N-O    16
Na-O-V     16
Li-Mn-O    16
Al-Mg      16
Name: system, dtype: int64

*Finding*: Mn-Na-O and Fe-Na-O are the most common ternaries. So, let's choose the Na-Fe-Mn-O quaternary as a hold-out. Also, there are a large number of Ti-O binary compounds, so let's use that one as the pairwise interaction holdout

## Exclude a Quaternary Diagram
Exclude the NaFeMnO data as the hold-out set

In [11]:
my_system = ["Na", "Fe", "Mn", "O"]

In [12]:
def get_test_data(elems):
    """Get the data that is in any of the phase diagrams that are subsets of a certain system
    
    Ex: For Na-Fe-O, these are Na-Fe-O, Na-Fe, Na-O, Fe-O, Na-Fe, Na, Fe, O
    
    :param elems: iterable of strs, phase diagram of interest
    :return: subset of OQMD in the constituent systems"""
    
    # Generate the constituent systems
    systems = set()
    for comb in product(*[elems,]*len(elems)):
        sys = "-".join(sorted(set(comb)))
        systems.add(sys)
    
    # Query for the data
    return oqmd_data.query(' or '.join('system=="%s"'%s for s in systems))

In [13]:
test_set = get_test_data(my_system)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Gathered a test set with 96 entries


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability,comp_obj,pretty_comp,nelems,system
357241,Mn4O5,-7.315486,11.4666,2.000249,0.000,-1.717088,0.347496,"(Mn, O)",Mn4O5,2,Mn-O
363586,Mn1Na2O3,-5.699281,10.5098,0.500015,1.632,-1.882803,-0.077266,"(Mn, Na, O)",Na2MnO3,3,Mn-Na-O
336692,Mn2Na4O5,-5.619245,12.0610,0.727329,0.738,-1.863454,-0.000989,"(Mn, Na, O)",Na4Mn2O5,3,Mn-Na-O
386530,Fe1O4,-5.033725,11.8043,0.000003,0.948,-0.208641,0.619132,"(Fe, O)",FeO4,2,Fe-O
363596,Mn4Na8O11,-5.649265,11.3401,0.608702,1.009,-1.863235,0.010312,"(Mn, Na, O)",Na8Mn4O11,3,Mn-Na-O


Measure the standard deviation and MAD of test set (useful for evaluating model performance)

In [14]:
mad = np.abs(test_set['delta_e'] - test_set['delta_e'].mean()).mean()
std = test_set['delta_e'].std()
print('MAD: {:.3f} eV/atom'.format(mad))
print('Std Dev: {:.3f} eV/atom'.format(std))

MAD: 0.792 eV/atom
Std Dev: 0.965 eV/atom


Remove these entries from the dataset at large

In [15]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Training set size is 275605 entries


### Save the data in Magpie-friendly format
We will be using Magpie to generate features

In [16]:
def save_magpie(data, path):
    """Save a dataframe in a magpie-friendly format
    
    :param data: pd.DataFrame, data to be saved
    :param path: str, output path"""
    
    data[['comp','delta_e']].to_csv(path, index=False, sep=' ')

In [17]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%(''.join(my_system))))

In [18]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%(''.join(my_system))))

## Generate the hold-pairwise out dataset

In [19]:
my_pair = ['Ti', 'O']

In [20]:
def get_test_data(elems):
    """Get the data that contains all of a certain set of elements.
        
    :param elems: iterable of strs, elems to exclude
    :return: subset of OQMD in the constituent systems"""
    
    # Process the dataset
    hit = oqmd_data['system'].apply(lambda x: all([e in x.split("-") for e in elems]))
    return oqmd_data[hit]

In [21]:
test_set = get_test_data(my_pair)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Gathered a test set with 561 entries


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability,comp_obj,pretty_comp,nelems,system
361895,Co2O5Ti2,-7.384125,11.2636,0.557531,0.000,-2.045327,0.501164,"(Co, O, Ti)",Ti2Co2O5,3,Co-O-Ti
144180,O3Ti1V1,-7.850653,11.2429,0.599952,0.000,-2.350284,0.563975,"(O, Ti, V)",TiVO3,3,O-Ti-V
338660,Li1O3Ti1,-7.083249,10.1925,0.200007,0.000,-2.470445,0.210412,"(Li, O, Ti)",LiTiO3,3,Li-O-Ti
33310,O2P1Ti3,-8.219338,11.4678,0.004111,0.000,-2.007836,0.281785,"(O, P, Ti)",Ti3PO2,3,O-P-Ti
346592,Ba2O4Ti1,-7.441456,17.6697,-0.000132,4.344,-3.210461,NaN,"(Ba, O, Ti)",Ba2TiO4,3,Ba-O-Ti


Measure the standard deviation and MAD of test set (useful for evaluating model performance)

In [22]:
mad = np.abs(test_set['delta_e'] - test_set['delta_e'].mean()).mean()
std = test_set['delta_e'].std()
print('MAD: {:.3f} eV/atom'.format(mad))
print('Std Dev: {:.3f} eV/atom'.format(std))

MAD: 0.478 eV/atom
Std Dev: 0.605 eV/atom


Remove these entries from the dataset at large

In [23]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Training set size is 275140 entries


Save the data

In [24]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%('-'.join(my_pair))))

In [25]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%('-'.join(my_pair))))